In [30]:
#MyConstructor
#MyEllipticCurveFactory

from sage.rings.integer_ring import ZZ
from sage.rings.rational_field import QQ
from sage.rings.rational_field import RationalField

import sage.rings.abc
from sage.rings.polynomial.multi_polynomial_ring import is_MPolynomialRing
from sage.rings.number_field.number_field_base import NumberField
from sage.rings.finite_rings.finite_field_base import FiniteField
from sage.rings.polynomial.multi_polynomial import MPolynomial
from sage.rings.ring import is_Ring

from sage.categories.fields import Fields
_Fields = Fields()


from sage.structure.sequence import Sequence
from sage.structure.element import parent, Expression
from sage.structure.factory import UniqueFactory

import sage.schemes.curves.projective_curve as plane_curve

from sage.misc.fast_methods import WithEqualityById

class MyEllipticCurveFactory(UniqueFactory):
    
    def create_key_and_extra_args(self, x=None, y=None, j=None, minimal_twist=True, **kwds):
        
        R = None
        if is_Ring(x):
            (R, x) = (x, y)

        if j is not None:
            if R is not None:
                try:
                    j = R(j)
                except (ZeroDivisionError, ValueError, TypeError):
                    raise ValueError("First parameter must be a ring containing %s" % j)
            elif x is not None:
                raise ValueError("First parameter (if present) must be a ring when j is specified")
            x = coefficients_from_j(j, minimal_twist)

        if isinstance(x, Expression) and x.is_relational():
            import operator
            if x.operator() != operator.eq:
                raise ValueError("no symbolic relations other than equalities are allowed")
            x = x.lhs() - x.rhs()

        if isinstance(parent(x), sage.rings.abc.SymbolicRing):
            x = x._polynomial_(QQ['x', 'y'])

        if isinstance(x, MPolynomial):
            if y is None:
                x = coefficients_from_Weierstrass_polynomial(x)
            else:
                # x is a cubic, y a rational point
                x = EllipticCurve_from_cubic(x, y, morphism=False).ainvs()

        if isinstance(x, str):
            # Interpret x as a Cremona or LMFDB label.
            from sage.databases.cremona import CremonaDatabase
            x, data = CremonaDatabase().coefficients_and_data(x)
            # User-provided keywords may override database entries.
            data.update(kwds)
            kwds = data

        if not isinstance(x, (list, tuple)):
            raise TypeError("invalid input to EllipticCurve constructor")

        if len(x) == 2:
            x = (0, 0, 0, x[0], x[1])
        elif len(x) != 5:
            raise ValueError("sequence of coefficients must have length 2 or 5")

        if R is None:
            R = Sequence(x).universe()
            if R in (ZZ, int):
                R = QQ

        return (R, tuple(R(a) for a in x)), kwds

    def create_object(self, version, key, **kwds):
        
        R, x = key

        if R is QQ:
            from sage.schemes.elliptic_curves.ell_rational_field import EllipticCurve_rational_field
            return EllipticCurve_rational_field(x, **kwds)
        elif isinstance(R, NumberField):
            from sage.schemes.elliptic_curves.ell_number_field import EllipticCurve_number_field
            return EllipticCurve_number_field(R, x)
        elif isinstance(R, sage.rings.abc.pAdicField):
            from sage.schemes.elliptic_curves.ell_padic_field import EllipticCurve_padic_field
            return EllipticCurve_padic_field(R, x)
        elif isinstance(R, FiniteField) or (isinstance(R, sage.rings.abc.IntegerModRing) and R.characteristic().is_prime()):
            from sage.schemes.elliptic_curves.ell_finite_field import EllipticCurve_finite_field
            return EllipticCurve_finite_field(R, x)
        elif R in _Fields:
            from sage.schemes.elliptic_curves.ell_field import EllipticCurve_field
            return EllipticCurve_field(R, x)
        
        return MyEllipticCurve_ring(R, x)


MyEllipticCurve = MyEllipticCurveFactory('sage.schemes.elliptic_curves.constructor.EllipticCurve')


In [31]:
def EllipticCurve_from_Weierstrass_polynomial(f):
    r"""
    Return the elliptic curve defined by a cubic in (long) Weierstrass
    form.

    INPUT:

    - ``f`` -- a inhomogeneous cubic polynomial in long Weierstrass
      form.

    OUTPUT: The elliptic curve defined by it.

    EXAMPLES::

        sage: R.<x,y> = QQ[]
        sage: f = y^2 + 1*x*y + 3*y - (x^3 + 2*x^2 + 4*x + 6)
        sage: EllipticCurve(f)
        Elliptic Curve defined by y^2 + x*y + 3*y = x^3 + 2*x^2 + 4*x + 6 over Rational Field
        sage: EllipticCurve(f).a_invariants()
        (1, 2, 3, 4, 6)

    The polynomial ring may have extra variables as long as they
    do not occur in the polynomial itself::

        sage: R.<x,y,z,w> = QQ[]
        sage: EllipticCurve(-y^2 + x^3 + 1)
        Elliptic Curve defined by y^2 = x^3 + 1 over Rational Field
        sage: EllipticCurve(-x^2 + y^3 + 1)
        Elliptic Curve defined by y^2 = x^3 + 1 over Rational Field
        sage: EllipticCurve(-w^2 + z^3 + 1)
        Elliptic Curve defined by y^2 = x^3 + 1 over Rational Field

    TESTS::

        sage: from sage.schemes.elliptic_curves.constructor import EllipticCurve_from_Weierstrass_polynomial
        sage: EllipticCurve_from_Weierstrass_polynomial(-w^2 + z^3 + 1)
        Elliptic Curve defined by y^2 = x^3 + 1 over Rational Field
    """
    return EllipticCurve(coefficients_from_Weierstrass_polynomial(f))

def coefficients_from_Weierstrass_polynomial(f):
    r"""
    Return the coefficients `[a_1, a_2, a_3, a_4, a_6]` of a cubic in
    Weierstrass form.

    EXAMPLES::

        sage: from sage.schemes.elliptic_curves.constructor import coefficients_from_Weierstrass_polynomial
        sage: R.<w,z> = QQ[]
        sage: coefficients_from_Weierstrass_polynomial(-w^2 + z^3 + 1)
        [0, 0, 0, 0, 1]
    """
    R = f.parent()
    cubic_variables = [ x for x in R.gens() if f.degree(x) == 3 ]
    quadratic_variables = [ y for y in R.gens() if f.degree(y) == 2 ]
    try:
        x = cubic_variables[0]
        y = quadratic_variables[0]
    except IndexError:
        raise ValueError('polynomial is not in long Weierstrass form')

    a1 = a2 = a3 = a4 = a6 = 0
    x3 = y2 = None
    for coeff, mon in f:
        if mon == x**3:
            x3 = coeff
        elif mon == x**2:
            a2 = coeff
        elif mon == x:
            a4 = coeff
        elif mon == 1:
            a6 = coeff
        elif mon == y**2:
            y2 = -coeff
        elif mon == x*y:
            a1 = -coeff
        elif mon == y:
            a3 = -coeff
        else:
            raise ValueError('polynomial is not in long Weierstrass form')

    if x3 != y2:
        raise ValueError('the coefficient of x^3 and -y^2 must be the same')
    elif x3 != 1:
        a1, a2, a3, a4, a6 = a1/x3, a2/x3, a3/x3, a4/x3, a6/x3
    return [a1, a2, a3, a4, a6]


def EllipticCurve_from_c4c6(c4, c6):
    r"""
    Return an elliptic curve with given `c_4` and
    `c_6` invariants.

    EXAMPLES::

        sage: E = EllipticCurve_from_c4c6(17, -2005)
        sage: E
        Elliptic Curve defined by y^2  = x^3 - 17/48*x + 2005/864 over Rational Field
        sage: E.c_invariants()
        (17, -2005)
    """
    try:
        K = c4.parent()
    except AttributeError:
        K = RationalField()
    if K not in _Fields:
        K = K.fraction_field()
    return EllipticCurve([-K(c4)/K(48), -K(c6)/K(864)])


def EllipticCurve_from_j(j, minimal_twist=True):
    r"""
    Return an elliptic curve with given `j`-invariant.

    INPUT:

    - ``j`` -- an element of some field.

    - ``minimal_twist`` (boolean, default True) -- If True and ``j``
      is in `\QQ`, the curve returned is a minimal twist, i.e. has
      minimal conductor; when there is more than one curve with
      minimal conductor, the curve returned is the one whose label
      comes first if the curves are in the CremonaDatabase, otherwise
      the one whose minimal a-invariants are first lexicographically.
      If `j` is not in `\QQ` this parameter is ignored.

    OUTPUT:

    An elliptic curve with `j`-invariant `j`.

    EXAMPLES::

        sage: E = EllipticCurve_from_j(0); E; E.j_invariant(); E.label()
        Elliptic Curve defined by y^2 + y = x^3 over Rational Field
        0
        '27a3'

        sage: E = EllipticCurve_from_j(1728); E; E.j_invariant(); E.label()
        Elliptic Curve defined by y^2 = x^3 - x over Rational Field
        1728
        '32a2'

        sage: E = EllipticCurve_from_j(1); E; E.j_invariant()
        Elliptic Curve defined by y^2 + x*y = x^3 + 36*x + 3455 over Rational Field
        1

    The ``minimal_twist`` parameter (ignored except over `\QQ` and
    True by default) controls whether or not a minimal twist is
    computed::

        sage: EllipticCurve_from_j(100)
        Elliptic Curve defined by y^2 = x^3 + x^2 + 3392*x + 307888 over Rational Field
        sage: _.conductor()
        33129800
        sage: EllipticCurve_from_j(100, minimal_twist=False)
        Elliptic Curve defined by y^2 = x^3 + 488400*x - 530076800 over Rational Field
        sage: _.conductor()
        298168200

    Since computing the minimal twist requires factoring both `j` and
    `j-1728` the following example would take a long time without
    setting ``minimal_twist`` to False::

       sage: E = EllipticCurve_from_j(2^256+1, minimal_twist=False)
       sage: E.j_invariant() == 2^256+1
       True
    """
    return EllipticCurve(coefficients_from_j(j, minimal_twist))


def coefficients_from_j(j, minimal_twist=True):
    r"""
    Return Weierstrass coefficients `(a_1, a_2, a_3, a_4, a_6)` for an
    elliptic curve with given `j`-invariant.

    INPUT: See :func:`EllipticCurve_from_j`.

    EXAMPLES::

        sage: from sage.schemes.elliptic_curves.constructor import coefficients_from_j
        sage: coefficients_from_j(0)
        [0, 0, 1, 0, 0]
        sage: coefficients_from_j(1728)
        [0, 0, 0, -1, 0]
        sage: coefficients_from_j(1)
        [1, 0, 0, 36, 3455]

    The ``minimal_twist`` parameter (ignored except over `\QQ` and
    True by default) controls whether or not a minimal twist is
    computed::

        sage: coefficients_from_j(100)
        [0, 1, 0, 3392, 307888]
        sage: coefficients_from_j(100, minimal_twist=False)
        [0, 0, 0, 488400, -530076800]
    """
    try:
        K = j.parent()
    except AttributeError:
        K = RationalField()
    if K not in _Fields:
        K = K.fraction_field()

    char = K.characteristic()
    if char == 2:
        if j == 0:
            return Sequence([0, 0, 1, 0, 0], universe=K)
        else:
            return Sequence([1, 0, 0, 0, 1/j], universe=K)
    if char == 3:
        if j == 0:
            return Sequence([0, 0, 0, 1, 0], universe=K)
        else:
            return Sequence([0, j, 0, 0, -j**2], universe=K)

    if K is RationalField():
        # we construct the minimal twist, i.e. the curve with minimal
        # conductor with this j_invariant:
        if j == 0:
            return Sequence([0, 0, 1, 0, 0], universe=K) # 27a3
        if j == 1728:
            return Sequence([0, 0, 0, -1, 0], universe=K) # 32a2

        if not minimal_twist:
            k = j-1728
            return Sequence([0, 0, 0, -3*j*k, -2*j*k**2], universe=K)

        n = j.numerator()
        m = n-1728*j.denominator()
        a4 = -3*n*m
        a6 = -2*n*m**2

        # Now E=[0,0,0,a4,a6] has j-invariant j=n/d
        from sage.sets.set import Set
        for p in Set(n.prime_divisors()+m.prime_divisors()):
            e = min(a4.valuation(p)//2,a6.valuation(p)//3)
            if e > 0:
                p = p**e
                a4 /= p**2
                a6 /= p**3

        # Now E=[0,0,0,a4,a6] is minimal at all p != 2,3
        tw = [-1,2,-2,3,-3,6,-6]
        E1 = EllipticCurve([0,0,0,a4,a6])
        Elist = [E1] + [E1.quadratic_twist(t) for t in tw]
        min_cond = min(E.conductor() for E in Elist)
        Elist = [E for E in Elist if E.conductor() == min_cond]
        if len(Elist) > 1:
            from sage.databases.cremona import CremonaDatabase, parse_cremona_label
            if min_cond <= CremonaDatabase().largest_conductor():
                sorter = lambda E: parse_cremona_label(E.label(), numerical_class_code=True)
            else:
                sorter = lambda E: E.ainvs()
            Elist.sort(key=sorter)
        return Sequence(Elist[0].ainvs())

    # defaults for all other fields:
    if j == 0:
        return Sequence([0, 0, 0, 0, 1], universe=K)
    if j == 1728:
        return Sequence([0, 0, 0, 1, 0], universe=K)
    k = j-1728
    return Sequence([0, 0, 0, -3*j*k, -2*j*k**2], universe=K)


def EllipticCurve_from_cubic(F, P=None, morphism=True):
    r"""
    Construct an elliptic curve from a ternary cubic with a rational point.

    If you just want the Weierstrass form and are not interested in
    the morphism then it is easier to use the function
    :func:`~sage.schemes.elliptic_curves.jacobian.Jacobian`
    instead. If there is a rational point on the given cubic, this
    function will construct the same elliptic curve but you do not have to
    supply the point ``P``.

    INPUT:

    - ``F`` -- a homogeneous cubic in three variables with rational
      coefficients, as a polynomial ring element, defining a smooth
      plane cubic curve `C`.

    - ``P`` -- a 3-tuple `(x,y,z)` defining a projective point on `C`,
      or ``None``.  If ``None`` then a rational flex will be used as a
      base point if one exists, otherwise an error will be raised.

    - ``morphism`` -- boolean (default: ``True``).  If ``True``
      returns a birational isomorphism from `C` to a Weierstrass
      elliptic curve `E`, otherwise just returns `E`.

    OUTPUT:

    Either (when ``morphism``=``False``) an elliptic curve `E` in long
    Weierstrass form isomorphic to the plane cubic curve `C` defined
    by the equation `F=0`.

    Or (when ``morphism=True``), a birational isomorphism from `C` to
    the elliptic curve `E`. If the given point is a flex, this is a
    linear isomorphism.

    .. NOTE::

        The function
        :func:`~sage.schemes.elliptic_curves.jacobian.Jacobian` may be
        used instead.  It constructs the same elliptic curve (which is in
        all cases the Jacobian of `(F=0)`) and needs no base point to be
        provided, but also returns no isomorphism since in general there
        is none: the plane cubic is only isomorphic to its Jacobian when
        it has a rational point.

    .. NOTE::

        When ``morphism=True``, a birational isomorphism between the
        curve `F=0` and the Weierstrass curve is returned. If the point
        happens to be a flex, then this is a linear isomorphism.  The
        morphism does not necessarily take the given point `P` to the
        point at infinity on `E`, since we always use a rational flex
        on `C` as base-point when one exists.

    EXAMPLES:

    First we find that the Fermat cubic is isomorphic to the curve
    with Cremona label 27a1::

        sage: R.<x,y,z> = QQ[]
        sage: cubic = x^3 + y^3 + z^3
        sage: P = [1,-1,0]
        sage: E = EllipticCurve_from_cubic(cubic, P, morphism=False); E
        Elliptic Curve defined by y^2 - 9*y = x^3 - 27 over Rational Field
        sage: E.cremona_label()
        '27a1'
        sage: EllipticCurve_from_cubic(cubic, [0,1,-1], morphism=False).cremona_label()
        '27a1'
        sage: EllipticCurve_from_cubic(cubic, [1,0,-1], morphism=False).cremona_label()
        '27a1'

    Next we find the minimal model and conductor of the Jacobian of the
    Selmer curve::

        sage: R.<a,b,c> = QQ[]
        sage: cubic = a^3 + b^3 + 60*c^3
        sage: P = [1,-1,0]
        sage: E = EllipticCurve_from_cubic(cubic, P, morphism=False);  E
        Elliptic Curve defined by y^2 - 540*y = x^3 - 97200 over Rational Field
        sage: E.minimal_model()
        Elliptic Curve defined by y^2 = x^3 - 24300 over Rational Field
        sage: E.conductor()
        24300

    We can also get the birational isomorphism to and from the
    Weierstrass form. We start with an example where ``P`` is a flex
    and the equivalence is a linear isomorphism::

        sage: f = EllipticCurve_from_cubic(cubic, P, morphism=True)
        sage: f
        Scheme morphism:
          From: Projective Plane Curve over Rational Field defined by a^3 + b^3 + 60*c^3
          To:   Elliptic Curve defined by y^2 - 540*y = x^3 - 97200 over Rational Field
          Defn: Defined on coordinates by sending (a : b : c) to
                (-c : 3*a : 1/180*a + 1/180*b)

        sage: finv = f.inverse();  finv
        Scheme morphism:
          From: Elliptic Curve defined by y^2 - 540*y = x^3 - 97200 over Rational Field
          To:   Projective Plane Curve over Rational Field defined by a^3 + b^3 + 60*c^3
          Defn: Defined on coordinates by sending (x : y : z) to
                (1/3*y : -1/3*y + 180*z : -x)

        Scheme morphism:
          From: Elliptic Curve defined by y^2 + 2*x*y + 20*y = x^3 - x^2 - 20*x - 400/3
                over Rational Field
          To:   Closed subscheme of Projective Space of dimension 2 over Rational Field
                defined by: a^3 + b^3 + 60*c^3
          Defn: Defined on coordinates by sending (x : y : z) to
                (x + y + 20*z : -x - y : -x)

    We verify that `f` maps the chosen point `P=(1,-1,0)` on the cubic
    to the origin of the elliptic curve::

        sage: f([1,-1,0])
        (0 : 1 : 0)
        sage: finv([0,1,0])
        (-1 : 1 : 0)

    To verify the output, we plug in the polynomials to check that
    this indeed transforms the cubic into Weierstrass form::

        sage: cubic(finv.defining_polynomials()) * finv.post_rescaling()
        -x^3 + y^2*z - 540*y*z^2 + 97200*z^3

        sage: E.defining_polynomial()(f.defining_polynomials()) * f.post_rescaling()
        a^3 + b^3 + 60*c^3

    If the given point is not a flex and the cubic has no rational
    flexes, then the cubic can not be transformed to a Weierstrass
    equation by a linear transformation. The general birational
    transformation is still a birational isomorphism, but is
    quadratic::

        sage: R.<x,y,z> = QQ[]
        sage: cubic = x^2*y + 4*x*y^2 + x^2*z + 8*x*y*z + 4*y^2*z + 9*x*z^2 + 9*y*z^2
        sage: f = EllipticCurve_from_cubic(cubic, [1,-1,1], morphism=True); f
        Scheme morphism:
          From: Projective Plane Curve over Rational Field defined
                by x^2*y + 4*x*y^2 + x^2*z + 8*x*y*z + 4*y^2*z + 9*x*z^2 + 9*y*z^2
          To:   Elliptic Curve defined
                by y^2 + 7560/19*x*y + 552960000000/2352637*y = x^3 - 3445200/133*x^2
                over Rational Field
          Defn: Defined on coordinates by sending (x : y : z) to
                (2527/17280*x^2 + 133/2160*x*y + 133/108000*y^2 + 133/2880*x*z
                   + 931/18000*y*z - 3857/48000*z^2
                 : -6859/288*x^2 + 323/36*x*y + 359/1800*y^2 + 551/48*x*z
                   + 2813/300*y*z + 24389/800*z^2
                 : -2352637/99532800000*x^2 - 2352637/124416000000*x*y
                   - 2352637/622080000000*y^2 + 2352637/82944000000*x*z
                   + 2352637/207360000000*y*z - 2352637/276480000000*z^2)

    Note that the morphism returned cannot be evaluated directly at
    the given point ``P=(1:-1:1)`` since the polynomials defining it
    all vanish there::

        sage: f([1,-1,1])
        Traceback (most recent call last):
        ...
        ValueError: [0, 0, 0] does not define a valid projective point since all entries are zero

    Using the group law on the codomain elliptic curve, which has rank
    1 and full 2-torsion, and the inverse morphism, we can find many
    points on the cubic.  First we find the preimages of multiples of
    the generator::

        sage: E = f.codomain()
        sage: E.label()
        '720e2'
        sage: E.rank()
        1
        sage: R = E.gens()[0]; R
        (-17280000/2527 : 9331200000/6859 : 1)
        sage: finv = f.inverse()
        sage: [finv(k*R) for k in range(1,10)]
        [(-4 : 1 : 0),
        (-1 : 4 : 1),
        (-20 : -55/76 : 1),
        (319/399 : -11339/7539 : 1),
        (159919/14360 : -4078139/1327840 : 1),
        (-27809119/63578639 : 1856146436/3425378659 : 1),
        (-510646582340/56909753439 : 424000923715/30153806197284 : 1),
        (-56686114363679/4050436059492161 : -2433034816977728281/1072927821085503881 : 1),
        (650589589099815846721/72056273157352822480 : -347376189546061993109881/194127383495944026752320 : 1)]

    The elliptic curve also has torsion, which we can map back::

        sage: E.torsion_points()
        [(0 : 1 : 0),
         (-144000000/17689 : 3533760000000/2352637 : 1),
         (-92160000/17689 : 2162073600000/2352637 : 1),
         (-5760000/17689 : -124070400000/2352637 : 1)]
        sage: [finv(Q) for Q in E.torsion_points() if Q]
        [(9 : -9/4 : 1), (-9 : 0 : 1), (0 : 1 : 0)]

    In this example, the given point ``P`` is not a flex but the cubic
    does have a rational flex, ``(-4:0:1)``.  We return a linear
    isomorphism which maps this flex to the point at infinity on the
    Weierstrass model::

        sage: R.<a,b,c> = QQ[]
        sage: cubic =  a^3 + 7*b^3 + 64*c^3
        sage: P = [2,2,-1]
        sage: f = EllipticCurve_from_cubic(cubic, P, morphism=True)
        sage: E = f.codomain();  E
        Elliptic Curve defined by y^2 - 258048*y = x^3 - 22196256768 over Rational Field
        sage: E.minimal_model()
        Elliptic Curve defined by y^2 + y = x^3 - 331 over Rational Field

        sage: f
        Scheme morphism:
          From: Projective Plane Curve over Rational Field defined by a^3 + 7*b^3 + 64*c^3
          To:   Elliptic Curve defined by y^2 - 258048*y = x^3 - 22196256768 over Rational Field
          Defn: Defined on coordinates by sending (a : b : c) to
                (b : -48*a : -1/5376*a - 1/1344*c)

        sage: finv = f.inverse();  finv
        Scheme morphism:
          From: Elliptic Curve defined by y^2 - 258048*y = x^3 - 22196256768 over Rational Field
          To:   Projective Plane Curve over Rational Field defined by a^3 + 7*b^3 + 64*c^3
          Defn: Defined on coordinates by sending (x : y : z) to
                (-1/48*y : x : 1/192*y - 1344*z)

        sage: cubic(finv.defining_polynomials()) * finv.post_rescaling()
        -x^3 + y^2*z - 258048*y*z^2 + 22196256768*z^3

        sage: E.defining_polynomial()(f.defining_polynomials()) * f.post_rescaling()
        a^3 + 7*b^3 + 64*c^3

        sage: f(P)
        (5376 : -258048 : 1)
        sage: f([-4,0,1])
        (0 : 1 : 0)

    It is possible to not provide a base point ``P`` provided that the
    cubic has a rational flex.  In this case the flexes will be found
    and one will be used as a base point::

        sage: R.<x,y,z> = QQ[]
        sage: cubic = x^3 + y^3 + z^3
        sage: f = EllipticCurve_from_cubic(cubic, morphism=True)
        sage: f
        Scheme morphism:
          From: Projective Plane Curve over Rational Field defined by x^3 + y^3 + z^3
          To:   Elliptic Curve defined by y^2 - 9*y = x^3 - 27 over Rational Field
          Defn: Defined on coordinates by sending (x : y : z) to
                (y : -3*x : -1/3*x - 1/3*z)

    An error will be raised if no point is given and there are no rational flexes::

        sage: R.<x,y,z> = QQ[]
        sage: cubic = 3*x^3 + 4*y^3 + 5*z^3
        sage: EllipticCurve_from_cubic(cubic)
        Traceback (most recent call last):
        ...
        ValueError: A point must be given when the cubic has no rational flexes

    An example over a finite field, using a flex::

        sage: K = GF(17)
        sage: R.<x,y,z> = K[]
        sage: cubic = 2*x^3 + 3*y^3 + 4*z^3
        sage: EllipticCurve_from_cubic(cubic, [0,3,1])
        Scheme morphism:
          From: Projective Plane Curve over Finite Field of size 17
                defined by 2*x^3 + 3*y^3 + 4*z^3
          To:   Elliptic Curve defined by y^2 + 16*y = x^3 + 11 over Finite Field of size 17
          Defn: Defined on coordinates by sending (x : y : z) to
                (-x : 4*y : 4*y + 5*z)

    An example in characteristic 3::

        sage: K = GF(3)
        sage: R.<x,y,z> = K[]
        sage: cubic = x^3 + y^3 + z^3 + x*y*z
        sage: EllipticCurve_from_cubic(cubic, [0,1,-1])
        Scheme morphism:
          From: Projective Plane Curve over Finite Field of size 3
                defined by x^3 + y^3 + x*y*z + z^3
          To:   Elliptic Curve defined by y^2 + x*y = x^3 + 1 over Finite Field of size 3
          Defn: Defined on coordinates by sending (x : y : z) to
                (y + z : -y : x)

    An example over a number field, using a non-flex and where there are no rational flexes::

        sage: # needs sage.rings.number_field
        sage: K.<a> = QuadraticField(-3)
        sage: R.<x,y,z> = K[]
        sage: cubic = 2*x^3 + 3*y^3 + 5*z^3
        sage: EllipticCurve_from_cubic(cubic, [1,1,-1])
        Scheme morphism:
          From: Projective Plane Curve over Number Field in a
                with defining polynomial x^2 + 3 with a = 1.732050807568878?*I
                defined by 2*x^3 + 3*y^3 + 5*z^3
          To:   Elliptic Curve defined by
                y^2 + 1754460/2053*x*y + 5226454388736000/8653002877*y
                = x^3 + (-652253285700/4214809)*x^2
                over Number Field in a with defining polynomial x^2 + 3
                with a = 1.732050807568878?*I
          Defn: Defined on coordinates by sending (x : y : z) to
                (-16424/127575*x^2 - 231989/680400*x*y - 14371/64800*y^2 - 26689/81648*x*z - 10265/27216*y*z - 2053/163296*z^2
                 : 24496/315*x^2 + 119243/840*x*y + 4837/80*y^2 + 67259/504*x*z + 25507/168*y*z + 5135/1008*z^2
                 : 8653002877/2099914709760000*x^2 + 8653002877/699971569920000*x*y + 8653002877/933295426560000*y^2 + 8653002877/419982941952000*x*z + 8653002877/279988627968000*y*z + 8653002877/335986353561600*z^2)

    An example over a function field, using a non-flex::

        sage: K.<t> = FunctionField(QQ)
        sage: R.<x,y,z> = K[]
        sage: cubic = x^3 + t*y^3 + (1+t)*z^3
        sage: EllipticCurve_from_cubic(cubic, [1,1,-1], morphism=False)                 # needs sage.libs.singular
        Elliptic Curve defined by y^2 + ((162*t^6+486*t^5+810*t^4+810*t^3+486*t^2+162*t)/(t^6+12*t^5-3*t^4-20*t^3-3*t^2+12*t+1))*x*y + ((314928*t^14+4094064*t^13+23462136*t^12+78102144*t^11+167561379*t^10+243026001*t^9+243026001*t^8+167561379*t^7+78102144*t^6+23462136*t^5+4094064*t^4+314928*t^3)/(t^14+40*t^13+577*t^12+3524*t^11+8075*t^10+5288*t^9-8661*t^8-17688*t^7-8661*t^6+5288*t^5+8075*t^4+3524*t^3+577*t^2+40*t+1))*y = x^3 + ((2187*t^12+13122*t^11-17496*t^10-207765*t^9-516132*t^8-673596*t^7-516132*t^6-207765*t^5-17496*t^4+13122*t^3+2187*t^2)/(t^12+24*t^11+138*t^10-112*t^9-477*t^8+72*t^7+708*t^6+72*t^5-477*t^4-112*t^3+138*t^2+24*t+1))*x^2
         over Rational function field in t over Rational Field

    TESTS:

    Here is a test for :issue:`21092`::

        sage: R.<x,y,z> = QQ[]
        sage: cubic = -3*x^2*y + 3*x*y^2 + 4*x^2*z + 4*y^2*z - 3*x*z^2 + 3*y*z^2 - 8*z^3
        sage: EllipticCurve_from_cubic(cubic, (-4/5, 4/5, 3/5) )
        Scheme morphism:
          From: Projective Plane Curve over Rational Field defined
                by -3*x^2*y + 3*x*y^2 + 4*x^2*z + 4*y^2*z - 3*x*z^2 + 3*y*z^2 - 8*z^3
          To:   Elliptic Curve defined by y^2 + 24*x*y + 3024*y = x^3 + 495*x^2 + 36288*x
                over Rational Field
          Defn: Defined on coordinates by sending (x : y : z) to
                (-1/3*z : 3*x : -1/1008*x + 1/1008*y + 1/378*z)
    """
    from sage.schemes.curves.constructor import Curve
    from sage.matrix.constructor import Matrix
    from sage.schemes.elliptic_curves.weierstrass_transform import \
        WeierstrassTransformationWithInverse

    # check the input
    R = F.parent()
    K = R.base_ring()
    if not is_MPolynomialRing(R):
        raise TypeError('equation must be a polynomial')
    if R.ngens() != 3 or F.nvariables() != 3:
        raise TypeError('equation must be a polynomial in three variables')
    if not F.is_homogeneous():
        raise TypeError('equation must be a homogeneous polynomial')

    C = Curve(F)
    if P:
        try:
            CP = C(P)
        except (TypeError, ValueError):
            raise TypeError('{} does not define a point on a projective curve over {} defined by {}'.format(P,K,F))

    x, y, z = R.gens()

    # Test whether P is a flex; if not test whether there are any rational flexes:

    hessian = Matrix([[F.derivative(v1, v2) for v1 in R.gens()] for v2 in R.gens()]).det()
    if P and hessian(P) == 0:
        flex_point = P
    else:
        flexes = C.intersection(Curve(hessian)).rational_points()
        if flexes:
            flex_point = list(flexes[0])
            if not P:
                P = flex_point
                CP = C(P)
        else:
            flex_point = None

    if flex_point is not None: # first case: base point is a flex
        P = flex_point
        L = tangent_at_smooth_point(C,P)
        dx, dy, dz = (L.coefficient(v) for v in R.gens())

        # find an invertible matrix M such that (0,1,0)M=P and
        # ML'=(0,0,1)' where L=[dx,dy,dx].  Then the linear transform
        # by M takes P to [0,1,0] and L to Z=0:

        if P[0]:
            Q1 = [0,-dz,dy]
            Q2 = [0,1,0] if dy else [0,0,1]
        elif P[1]:
            Q1 = [dz,0,-dx]
            Q2 = [1,0,0] if dx else [0,0,1]
        else:
            Q1 = [-dy,dx,0]
            Q2 = [1,0,0] if dx else [0,1,0]

        M = Matrix(K,[Q1,P,Q2])
        # assert M.is_invertible()
        # assert list(vector([0,1,0])*M) == P
        # assert list(M*vector([dx,dy,dz]))[:2] == [0,0]

        M = M.transpose()
        F2 = R(M.act_on_polynomial(F))

        # scale and dehomogenise
        a = K(F2.coefficient(x**3))
        b = K(F2.coefficient(y*y*z))

        F3 = F2([-x, y/b, z*a*b]) / a
        # assert F3.coefficient(x**3) == -1
        # assert F3.coefficient(y*y*z) == 1
        E = EllipticCurve(F3([x,y,1]))
        if not morphism:
            return E

        # Construct the (linear) morphism
        M = M * Matrix(K,[[-1,0,0],[0,1/b,0],[0,0,a*b]])
        inv_defining_poly = [ M[i,0]*x + M[i,1]*y + M[i,2]*z for i in range(3) ]
        inv_post = 1/a
        M = M.inverse()
        fwd_defining_poly = [ M[i,0]*x + M[i,1]*y + M[i,2]*z for i in range(3) ]
        fwd_post = a

    else: # Second case: no flexes
        if not P:
            raise ValueError('A point must be given when the cubic has no rational flexes')
        L = tangent_at_smooth_point(C,P)
        Qlist = [Q for Q in C.intersection(Curve(L)).rational_points() if C(Q) != CP]
        # assert Qlist
        P2 = C(Qlist[0])
        L2 = tangent_at_smooth_point(C,P2)
        Qlist = [Q for Q in C.intersection(Curve(L2)).rational_points() if C(Q) != P2]
        # assert Qlist
        P3 = C(Qlist[0])

        # NB This construction of P3 relies on P2 not being a flex.
        # If we want to use a non-flex as P when there are rational
        # flexes this would be a problem.  However, the only condition
        # which P3 must satisfy is that it is on the tangent at P2, it
        # need not lie on the cubic.

        # send P, P2, P3 to (1:0:0), (0:1:0), (0:0:1) respectively
        M = Matrix(K, [P, list(P2), list(P3)]).transpose()
        F2 = M.act_on_polynomial(F)
        xyzM = [ M[i,0]*x + M[i,1]*y + M[i,2]*z for i in range(3) ]
        # assert F(xyzM)==F2

        # substitute x = U^2, y = V*W, z = U*W, and rename (x,y,z)=(U,V,W)
        T1 = [x*x,y*z,x*z]
        S1 = x**2*z
        F3 = F2(T1) // S1
        xyzC = [ t(T1) for t in xyzM ]
        # assert F3 == F(xyzC) // S1

        # scale and dehomogenise
        a = K(F3.coefficient(x**3))
        b = K(F3.coefficient(y*y*z))
        ab = a*b

        T2 = [-x, y/b, ab*z]
        F4 = F3(T2) / a
        # assert F4.coefficient(x**3) == -1
        # assert F4.coefficient(y*y*z) == 1
        xyzW = [ t(T2) for t in xyzC ]
        S2 = a*S1(T2)
        # assert F4 == F(xyzW) // S2

        E = EllipticCurve(F4([x,y,1]))
        if not morphism:
            return E

        inv_defining_poly = xyzW
        inv_post = 1/S2
        # assert F4==F(inv_defining_poly)*inv_post
        MI = M.inverse()
        xyzI = [ (MI[i,0]*x + MI[i,1]*y + MI[i,2]*z) for i in range(3) ]
        T1I = [x*z,x*y,z*z] # inverse of T1
        xyzIC = [ t(xyzI) for t in T1I ]
        T2I = [-x, b*y, z/ab] # inverse of T2
        xyzIW = [ t(xyzIC) for t in T2I ]
        fwd_defining_poly = xyzIW
        fwd_post = a/(x*z*z)(xyzI)
        # assert F4(fwd_defining_poly)*fwd_post == F

    # Construct the morphism

    return WeierstrassTransformationWithInverse(
        C, E, fwd_defining_poly, fwd_post, inv_defining_poly, inv_post)


def tangent_at_smooth_point(C,P):
    r"""Return the tangent at the smooth point `P` of projective curve `C`.

    INPUT:

    - ``C`` -- a projective plane curve.

    - ``P`` -- a 3-tuple `(x,y,z)` defining a projective point on `C`.

    OUTPUT:

    The linear form defining the tangent at `P` to `C`.

    EXAMPLES::

        sage: R.<x,y,z> = QQ[]
        sage: from sage.schemes.elliptic_curves.constructor import tangent_at_smooth_point
        sage: C = Curve(x^3 + y^3 + 60*z^3)
        sage: tangent_at_smooth_point(C, [1,-1,0])
        x + y

        sage: K.<t> = FunctionField(QQ)
        sage: R.<x,y,z> = K[]
        sage: C = Curve(x^3 + 2*y^3 + 3*z^3)
        sage: from sage.schemes.elliptic_curves.constructor import tangent_at_smooth_point
        sage: tangent_at_smooth_point(C,[1,1,-1])
        3*x + 6*y + 9*z
    """
    # Over function fields such as QQ(t) an error is raised with the
    # default (factor=True).  Note that factor=False returns the
    # product of the tangents in case of a multiple point, while here
    # `P` is assumed smooth so factorization is unnecessary, but over
    # QQ (for example) including the factorization gives better
    # results, for example returning x+y instead of 3x+3y in the
    # doctest.
    try:
        return C.tangents(P)[0]
    except NotImplementedError:
        return C.tangents(P,factor=False)[0]

def chord_and_tangent(F, P):
    r"""Return the third point of intersection of a cubic with the tangent at one point.

    INPUT:

    - ``F`` -- a homogeneous cubic in three variables with rational
      coefficients, as a polynomial ring element, defining a smooth
      plane cubic curve.

    - ``P`` -- a 3-tuple `(x,y,z)` defining a projective point on the
      curve `F=0`.

    OUTPUT:

    A point ``Q`` such that ``F(Q)=0``, namely the third point of
    intersection of the tangent at ``P`` with the curve ``F=0``, so
    ``Q=P`` if and only if ``P`` is a flex.

    EXAMPLES::

        sage: R.<x,y,z> = QQ[]
        sage: from sage.schemes.elliptic_curves.constructor import chord_and_tangent
        sage: F = x^3 + y^3 + 60*z^3
        sage: chord_and_tangent(F, [1,-1,0])
        (-1 : 1 : 0)

        sage: F = x^3 + 7*y^3 + 64*z^3
        sage: p0 = [2,2,-1]
        sage: p1 = chord_and_tangent(F, p0);  p1
        (5 : -3 : 1)
        sage: p2 = chord_and_tangent(F, p1);  p2
        (-1265/314 : 183/314 : 1)

    TESTS::

        sage: F(list(p2))
        0
        sage: list(map(type, p2))
        [<... 'sage.rings.rational.Rational'>,
         <... 'sage.rings.rational.Rational'>,
         <... 'sage.rings.rational.Rational'>]

    See :issue:`16068`::

        sage: F = x**3 - 4*x**2*y - 65*x*y**2 + 3*x*y*z - 76*y*z**2
        sage: chord_and_tangent(F, [0, 1, 0])
        (0 : 0 : 1)
    """
    from sage.schemes.curves.constructor import Curve
    # check the input
    R = F.parent()
    if not is_MPolynomialRing(R):
        raise TypeError('equation must be a polynomial')
    if R.ngens() != 3:
        raise TypeError('{} is not a polynomial in three variables'.format(F))
    if not F.is_homogeneous():
        raise TypeError('{} is not a homogeneous polynomial'.format(F))
    x, y, z = R.gens()
    if len(P) != 3:
        raise TypeError('{} is not a projective point'.format(P))
    K = R.base_ring()
    try:
        C = Curve(F)
        P = C(P)
    except (TypeError, ValueError):
        raise TypeError('{} does not define a point on a projective curve over {} defined by {}'.format(P,K,F))

    L = Curve(tangent_at_smooth_point(C,P))
    Qlist = [Q for Q in C.intersection(L).rational_points() if Q != P]
    if Qlist:
        return Qlist[0]
    return P


def projective_point(p):
    r"""
    Return equivalent point with denominators removed

    INPUT:

    - ``P``, ``Q`` -- list/tuple of projective coordinates.

    OUTPUT:

    List of projective coordinates.

    EXAMPLES::

        sage: from sage.schemes.elliptic_curves.constructor import projective_point
        sage: projective_point([4/5, 6/5, 8/5])
        [2, 3, 4]
        sage: F = GF(11)
        sage: projective_point([F(4), F(8), F(2)])
        [4, 8, 2]
    """
    from sage.rings.integer import GCD_list
    from sage.arith.functions import LCM_list
    try:
        p_gcd = GCD_list([x.numerator() for x in p])
        p_lcm = LCM_list(x.denominator() for x in p)
    except AttributeError:
        return p
    scale = p_lcm / p_gcd
    return [scale * x for x in p]


def are_projectively_equivalent(P, Q, base_ring):
    r"""
    Test whether ``P`` and ``Q`` are projectively equivalent.

    INPUT:

    - ``P``, ``Q`` -- list/tuple of projective coordinates.

    - ``base_ring`` -- the base ring.

    OUTPUT: A boolean.

    EXAMPLES::

        sage: from sage.schemes.elliptic_curves.constructor import are_projectively_equivalent
        sage: are_projectively_equivalent([0,1,2,3], [0,1,2,2], base_ring=QQ)
        False
        sage: are_projectively_equivalent([0,1,2,3], [0,2,4,6], base_ring=QQ)
        True
    """
    from sage.matrix.constructor import matrix
    return matrix(base_ring, [P, Q]).rank() < 2


def EllipticCurves_with_good_reduction_outside_S(S=[], proof=None, verbose=False):
    r"""
    Return a sorted list of all elliptic curves defined over `\QQ`
    with good reduction outside the set `S` of primes.

    INPUT:

    - ``S`` -- list of primes (default: empty list)

    - ``proof`` -- boolean (default ``True``): the MW basis for
      auxiliary curves will be computed with this proof flag

    - ``verbose`` -- boolean (default ``False``): if ``True``, some details
      of the computation will be output

    .. NOTE::

        Proof flag: The algorithm used requires determining all
        S-integral points on several auxiliary curves, which in turn
        requires the computation of their generators.  This is not
        always possible (even in theory) using current knowledge.

        The value of this flag is passed to the function which
        computes generators of various auxiliary elliptic curves, in
        order to find their S-integral points.  Set to ``False`` if the
        default (``True``) causes warning messages, but note that you can
        then not rely on the set of curves returned being
        complete.

    EXAMPLES::

        sage: EllipticCurves_with_good_reduction_outside_S([])
        []
        sage: elist = EllipticCurves_with_good_reduction_outside_S([2])
        sage: elist
        [Elliptic Curve defined by y^2 = x^3 + 4*x over Rational Field,
         Elliptic Curve defined by y^2 = x^3 - x over Rational Field,
         ...
         Elliptic Curve defined by y^2 = x^3 - x^2 - 13*x + 21 over Rational Field]
        sage: len(elist)
        24
        sage: ', '.join(e.label() for e in elist)
        '32a1, 32a2, 32a3, 32a4, 64a1, 64a2, 64a3, 64a4, 128a1, 128a2, 128b1, 128b2, 128c1, 128c2, 128d1, 128d2, 256a1, 256a2, 256b1, 256b2, 256c1, 256c2, 256d1, 256d2'

    Without ``Proof=False``, this example gives two warnings::

        sage: elist = EllipticCurves_with_good_reduction_outside_S([11], proof=False)   # long time (14s on sage.math, 2011)
        sage: len(elist)                                                                # long time
        12
        sage: ', '.join(e.label() for e in elist)                                       # long time
        '11a1, 11a2, 11a3, 121a1, 121a2, 121b1, 121b2, 121c1, 121c2, 121d1, 121d2, 121d3'

        sage: # long time
        sage: elist = EllipticCurves_with_good_reduction_outside_S([2,3])               # long time (26s on sage.math, 2011)
        sage: len(elist)
        752
        sage: conds = sorted(set([e.conductor() for e in elist]))
        sage: max(conds)
        62208
        sage: [N.factor() for N in conds]
        [2^3 * 3,
         3^3,
         2^5,
         2^2 * 3^2,
         2^4 * 3,
         2 * 3^3,
         2^6,
         2^3 * 3^2,
         2^5 * 3,
         2^2 * 3^3,
         2^7,
         2^4 * 3^2,
         2 * 3^4,
         2^6 * 3,
         2^3 * 3^3,
         3^5,
         2^8,
         2^5 * 3^2,
         2^2 * 3^4,
         2^7 * 3,
         2^4 * 3^3,
         2 * 3^5,
         2^6 * 3^2,
         2^3 * 3^4,
         2^8 * 3,
         2^5 * 3^3,
         2^2 * 3^5,
         2^7 * 3^2,
         2^4 * 3^4,
         2^6 * 3^3,
         2^3 * 3^5,
         2^8 * 3^2,
         2^5 * 3^4,
         2^7 * 3^3,
         2^4 * 3^5,
         2^6 * 3^4,
         2^8 * 3^3,
         2^5 * 3^5,
         2^7 * 3^4,
         2^6 * 3^5,
         2^8 * 3^4,
         2^7 * 3^5,
         2^8 * 3^5]
    """
    from sage.schemes.elliptic_curves.ell_egros import egros_from_jlist, egros_get_j
    return egros_from_jlist(egros_get_j(S, proof=proof, verbose=verbose), S)

In [32]:
MyEllipticCurve(QQ,[1, 2])

Elliptic Curve defined by y^2 = x^3 + x + 2 over Rational Field

In [33]:
MyEllipticCurve([1, 2])

Elliptic Curve defined by y^2 = x^3 + x + 2 over Rational Field

In [34]:
MyEllipticCurve(QQ, j=5)

Elliptic Curve defined by y^2 + x*y = x^3 + x^2 + 180*x + 17255 over Rational Field

In [35]:
x,y = var('x,y')

In [36]:
y^2 == x^3 +1

y^2 == x^3 + 1

In [37]:
MyEllipticCurve(y^2 == x^3 +1)

Elliptic Curve defined by y^2 = x^3 + 1 over Rational Field

In [38]:
#Cremona’s tables of elliptic curves
MyEllipticCurve("17a")

Elliptic Curve defined by y^2 + x*y + y = x^3 - x^2 - x - 14 over Rational Field

In [39]:
#Myell_generic
#more import

import math
from sage.arith.misc import valuation

import sage.rings.abc
from sage.rings.finite_rings.integer_mod import mod
from sage.rings.polynomial.polynomial_ring_constructor import PolynomialRing
from sage.rings.polynomial.polynomial_ring import polygen, polygens
from sage.rings.polynomial.polynomial_element import polynomial_is_variable
from sage.rings.polynomial.polynomial_quotient_ring_element import PolynomialQuotientRingElement
from sage.rings.finite_rings.finite_field_base import FiniteField
import sage.groups.additive_abelian.additive_abelian_group as groups
import sage.groups.generic as generic

from sage.arith.functions import lcm
from sage.rings.integer import Integer
from sage.rings.rational import Rational
from sage.rings.finite_rings.finite_field_constructor import FiniteField as GF
from sage.rings.rational_field import RationalField
from sage.rings.real_mpfr import RealField
from sage.misc.cachefunc import cached_method


# Schemes
import sage.schemes.projective.projective_space as projective_space
from sage.schemes.projective.projective_homset import SchemeHomset_points_projective_ring


# From sage.schemes.elliptic_curves.
from sage.schemes.elliptic_curves import ell_point
from sage.schemes.elliptic_curves import ell_torsion
from sage.schemes.elliptic_curves import constructor
from sage.schemes.elliptic_curves import formal_group
from sage.schemes.elliptic_curves import weierstrass_morphism as wm


sqrt = math.sqrt
exp = math.exp

In [40]:
#__contains__ has been changed
#is_x_coord lift_x are not implemented 
#is_on_curve has been changed. from E.is_on_curve(x,y) to E.is_on_curve(x,y,z=1)

#rst_transform and functions related to isomorphism are not implemented
#pari_curve __pari
#formal group


from sage.schemes.projective.projective_point import SchemeMorphism_point_projective_ring


def is_EllipticCurve(x):
    
    return isinstance(x, EllipticCurve_generic)

class MyEllipticCurvePoint_ring(SchemeMorphism_point_projective_ring):
    pass

class MyEllipticCurve_generic(WithEqualityById, plane_curve.ProjectivePlaneCurve):
    
    def __init__(self, K, ainvs, category=None):
        
        self.__base_ring = K
        self.__ainvs = tuple(K(a) for a in ainvs)
        if self.discriminant() == 0:
            raise ArithmeticError(self._equation_string() + " defines a singular curve")
        PP = projective_space.ProjectiveSpace(2, K, names='xyz')
        x, y, z = PP.coordinate_ring().gens()
        a1, a2, a3, a4, a6 = ainvs
        f = y**2*z + (a1*x + a3*z)*y*z \
            - (x**3 + a2*x**2*z + a4*x*z**2 + a6*z**3)
        plane_curve.ProjectivePlaneCurve.__init__(self, PP, f, category=category)

        self.__divpolys = ({}, {}, {})

    _point = MyEllipticCurvePoint_ring

    def _defining_params_(self):
        
        return (self.__base_ring, list(self.__ainvs))

    def _equation_string(self):
        
        b = self.ainvs()
        a = [z._coeff_repr() for z in b]
        s = "y^2"
        if a[0] == "-1":
            s += " - x*y"
        elif a[0] == '1':
            s += " + x*y"
        elif b[0]:
            s += " + %s*x*y" % a[0]
        if a[2] == "-1":
            s += " - y"
        elif a[2] == '1':
            s += " + y"
        elif b[2]:
            s += " + %s*y" % a[2]
        s += " = x^3"
        if a[1] == "-1":
            s += " - x^2"
        elif a[1] == '1':
            s += " + x^2"
        elif b[1]:
            s += " + %s*x^2" % a[1]
        if a[3] == "-1":
            s += " - x"
        elif a[3] == '1':
            s += " + x"
        elif b[3]:
            s += " + %s*x" % a[3]
        if a[4] == '-1':
            s += " - 1"
        elif a[4] == '1':
            s += " + 1"
        elif b[4]:
            s += " + %s" % a[4]
        return s.replace("+ -","- ")

    def _repr_(self):
        
        s = "Elliptic Curve defined by "
        s += self._equation_string()
        s += " over %s" % self.base_ring()
        return s

    def _latex_(self):
        
        from sage.rings.polynomial.polynomial_ring import polygen
        a = self.ainvs()
        x, y = polygen(self.base_ring(), 'x, y')
        s = "y^2"
        if a[0] or a[2]:
            s += " + " + (a[0]*x*y + a[2]*y)._latex_()
        s += " = "
        s += (x**3 + a[1]*x**2 + a[3]*x + a[4])._latex_()
        s += " "
        s = s.replace("+ -","- ")
        return s

    def _pari_init_(self):
        
        return 'ellinit([%s])' % (','.join(x._pari_init_()
                                           for x in self.ainvs()))

    def _magma_init_(self, magma):
        
        kmn = magma(self.base_ring())._ref()
        return 'EllipticCurve([%s|%s])' % (kmn,','.join(x._magma_init_(magma)
                                                        for x in self.ainvs()))

    def _symbolic_(self, SR):
        
        a = [SR(x) for x in self.a_invariants()]
        x, y = SR.var('x, y')
        return y**2 + a[0]*x*y + a[2]*y == x**3 + a[1]*x**2 + a[3]*x + a[4]

    def __contains__(self, P):
        
        if not isinstance(P, ell_point.EllipticCurvePoint):
            try:
                P = self(P)
            except TypeError:
                return False
        if P.curve() == self:
            return True
        x, y, z, a = P[0], P[1], P[2], self.ainvs()
        return y**2*z + a[0]*x*y*z  + a[2]*y*z**2 == x**3 + a[1]*x**2*z + a[3]*x*z**2 + a[4]*z**3

    def __call__(self, *args, **kwds):
        
        if len(args) == 1 and args[0] == 0:
            R = self.base_ring()
            return self.point([R(0),R(1),R(0)], check=False)
        P = args[0]
        if isinstance(P, groups.AdditiveAbelianGroupElement) and isinstance(P.parent(),ell_torsion.EllipticCurveTorsionSubgroup):
            return self(P.element())

        return plane_curve.ProjectivePlaneCurve.__call__(self, *args, **kwds)

    def _reduce_point(self, R, p):
        r"""
        Reduces a point R on an elliptic curve to the corresponding point on
        the elliptic curve reduced modulo p.

        Used to coerce points between
        curves when p is a factor of the denominator of one of the
        coordinates.
        """
        if R.is_zero():
            return R.curve().change_ring(GF(p))(0)
        x, y, z = R.xyz()
        d = lcm(x.denominator(), y.denominator(), z.denominator())
        return R.curve().change_ring(GF(p))([x*d, y*d, z*d])

    '''
    def is_x_coord(self, x):
        r"""
        Return True if ``x`` is the `x`-coordinate of a point on this curve.

        .. NOTE::

            See also :meth:`lift_x` to find the point(s) with a given
            `x`-coordinate.  This function may be useful in cases where
            testing an element of the base field for being a square is
            faster than finding its square root.

        """
        K = self.base_ring()
        try:
            x = K(x)
        except TypeError:
            raise TypeError('x must be coercible into the base ring of the curve')
        a1, a2, a3, a4, a6 = self.ainvs()
        fx = ((x + a2) * x + a4) * x + a6
        if a1.is_zero() and a3.is_zero():
            return fx.is_square()
        b = (a1*x + a3)
        if K.characteristic() == 2:
            R = PolynomialRing(K, 'y')
            F = R([-fx,b,1])
            return bool(F.roots())
        D = b*b + 4*fx
        return D.is_square()
    '''

    '''
    def lift_x(self, x, all=False, extend=False):
        r"""
        Return one or all points with given `x`-coordinate.

        This method is deterministic: It returns the same data each
        time when called again with the same `x`.

        """
        K = self.base_ring()
        L = x.parent()
        E = self

        # Check that the x-coordinate is in K and extend otherwise if possible:
        phi = K.coerce_map_from(L)
        if phi:
            x = phi(x)
            L = K  # new parent of x
        else:
            if L.coerce_map_from(K):
                E = E.change_ring(L)
                L = E.base_ring()
                x = L(x)
            else:
                raise TypeError("Unable to construct a point with x in {} over {}".format(L,K))

        # Now E is defined over L, possibly an extension of K, and x is in L

        a1, a2, a3, a4, a6 = E.ainvs()
        b = (a1*x + a3)
        f = ((x + a2) * x + a4) * x + a6

        # If possible find the associated y coorindates in L:

        if K.characteristic() == 2:
            R = PolynomialRing(L, 'y')
            F = R([-f,b,1])
            ys = F.roots(L, multiplicities=False)
        else:
            D = b*b+4*f
            ys = []
            if D.is_square():  # avoid automatic creation of sqrts
                ys = [(-b+d)/2 for d in D.sqrt(all=True)]

        ys.sort()  # ensure deterministic behavior

        # Return the point(s) if any:

        if ys:
            one = L.one()
            if all:
                return [E.point([x, y, one], check=False) for y in ys]
            else:
                return E.point([x, ys[0], one], check=False)

        # otherwise if the additional extension was not requested return the empty list or raise an error:

        if not extend:
            if all:
                return []
            else:
                raise ValueError("No point with x-coordinate {} on {}".format(x, self))

        # Now make the extension needed to contain the y-coordinates:

        if K.characteristic() != 2:  # else we already defined F
            R = PolynomialRing(L, 'y')
            F = R([-f,b,1])
        M = L.fraction_field().extension(F, names='y')
        EM = E.change_ring(M)
        y1 = M.gen()
        y2 = -b-y1
        if y2 == y1:
            ys = [y1]
        else:
            ys = [y1, y2]
        ys.sort()  # ensure deterministic behavior
        x = M(x)
        one = M.one()
        if all:
            return [EM.point([x, y, one], check=False) for y in ys]
        else:
            return EM.point([x, ys[0], one], check=False)
    '''
    
    def _point_homset(self, *args, **kwds):
        r"""
        Internal function. Return the (abstract) group of points on this
        elliptic curve over a ring.
        """
        
        return SchemeHomset_points_projective_ring(*args, **kwds)

    def __getitem__(self, n):
        r"""
        Placeholder for standard indexing function.

        EXAMPLES::

            sage: E = EllipticCurve(QQ,[1,1])
            sage: E[2]
            Traceback (most recent call last):
            ...
            NotImplementedError: not implemented.
        """
        raise NotImplementedError("not implemented.")

    def __is_over_RationalField(self):
        r"""
        Internal function. Return true iff the base ring of this elliptic
        curve is the field of rational numbers.

        EXAMPLES::

            sage: E = EllipticCurve(QQ,[1,1])
            sage: E._EllipticCurve_generic__is_over_RationalField()
            True
            sage: E = EllipticCurve(GF(5),[1,1])
            sage: E._EllipticCurve_generic__is_over_RationalField()
            False
        """
        return isinstance(self.base_ring(), RationalField)

    def is_on_curve(self, x, y, z=1):
        r"""
        Return True if `(x,y)` is an affine point on this curve.

        INPUT:

        - ``x``, ``y`` -- elements of the base ring of the curve.

        EXAMPLES::

            sage: E = EllipticCurve(QQ,[1,1])
            sage: E.is_on_curve(0,1)
            True
            sage: E.is_on_curve(1,1)
            False
        """
        a = self.ainvs()
        return y**2*z + a[0]*x*y*z + a[2]*y*z**2 == x**3 + a[1]*x**2*z + a[3]*x*z**2 + a[4]*z**3

    def a_invariants(self):
        
        return self.__ainvs

    ainvs = a_invariants

    def a1(self):
        
        return self.__ainvs[0]

    def a2(self):
        
        return self.__ainvs[1]

    def a3(self):
        
        return self.__ainvs[2]

    def a4(self):
        
        return self.__ainvs[3]

    def a6(self):
        
        return self.__ainvs[4]

    @cached_method
    def b_invariants(self):
        
        a1, a2, a3, a4, a6 = self.ainvs()
        return (a1*a1 + 4*a2,
                a1*a3 + 2*a4,
                a3**2 + 4*a6,
                a1**2 * a6 + 4*a2*a6 - a1*a3*a4 + a2*a3**2 - a4**2)

    def b2(self):
        
        return self.b_invariants()[0]

    def b4(self):
        
        return self.b_invariants()[1]

    def b6(self):
        
        return self.b_invariants()[2]

    def b8(self):
        
        return self.b_invariants()[3]

    @cached_method
    def c_invariants(self):
        
        b2, b4, b6, b8 = self.b_invariants()
        # note: c6 is wrong in Silverman, but right in Cremona
        return (b2**2 - 24*b4,
                -b2**3 + 36*b2*b4 - 216*b6)

    def c4(self):
        
        return self.c_invariants()[0]

    def c6(self):
        
        return self.c_invariants()[1]

    @cached_method
    def discriminant(self):
        
        b2, b4, b6, b8 = self.b_invariants()
        return -b2**2*b8 - 8*b4**3 - 27*b6**2 + 9*b2*b4*b6

    @cached_method
    def j_invariant(self):
        
        c4, _ = self.c_invariants()
        return c4**3 / self.discriminant()

    def base_extend(self, R):
        r"""
        Return the base extension of ``self`` to `R`.

        INPUT:

        - ``R`` -- either a ring into which the `a`-invariants of
          ``self`` may be converted, or a morphism which may be
          applied to them.

        OUTPUT:

        An elliptic curve over the new ring whose `a`-invariants are
        the images of the `a`-invariants of ``self``.

        EXAMPLES::

            sage: E = EllipticCurve(GF(5), [1,1]); E
            Elliptic Curve defined by y^2 = x^3 + x + 1 over Finite Field of size 5
            sage: E1 = E.base_extend(GF(125,'a')); E1                                   # needs sage.rings.finite_rings
            Elliptic Curve defined by y^2 = x^3 + x + 1 over Finite Field in a of size 5^3

        TESTS:

        Check that we are correctly keeping track of known
        cardinalities when extending the base field::

            sage: # needs sage.rings.finite_rings
            sage: E = EllipticCurve(j=GF(7)(5))
            sage: E.cardinality()
            10
            sage: EE = E.base_extend(GF(7^2))
            sage: EE._order
            60

        Changing to a smaller field should not cache orders::

            sage: EE = EllipticCurve(j=GF(7^3)(6))                                      # needs sage.rings.finite_rings
            sage: hasattr(EE.change_ring(GF(7)), '_order')                              # needs sage.rings.finite_rings
            False

        Changing to a field of different characteristic should
        not cache orders::

            sage: Elift = E.change_ring(QQ)                                             # needs sage.rings.finite_rings
            sage: hasattr(Elift, '_order')                                              # needs sage.rings.finite_rings
            False
        """
        E = constructor.EllipticCurve([R(a) for a in self.a_invariants()])

        if isinstance(R, FiniteField) and hasattr(self, '_order') and self.__base_ring.is_subring(R):
            # The cardinality over an extension field follows easily
            # from the cardinality over the smaller field.
            n = R.cardinality().log(self.__base_ring.cardinality())
            E._order = self.cardinality(extension_degree=n)

        return E

    def change_ring(self, R):
        """
        Return the base change of ``self`` to `R`.

        This has the same effect as ``self.base_extend(R)``.

        EXAMPLES::

            sage: # needs sage.rings.finite_rings
            sage: F2 = GF(5^2,'a'); a = F2.gen()
            sage: F4 = GF(5^4,'b'); b = F4.gen()
            sage: roots = a.charpoly().roots(ring=F4, multiplicities=False)
            sage: h = F2.hom([roots[0]], F4)
            sage: E = EllipticCurve(F2, [1,a]); E
            Elliptic Curve defined by y^2 = x^3 + x + a
            over Finite Field in a of size 5^2
            sage: E.change_ring(h)
            Elliptic Curve defined by y^2 = x^3 + x + (4*b^3+4*b^2+4*b+3)
            over Finite Field in b of size 5^4
        """
        return self.base_extend(R)

    def base_ring(self):
        
        return self.__base_ring

    def gens(self):
        r"""
        Placeholder function to return generators of an elliptic curve.

        """
        raise NotImplementedError("not implemented.")

    def gen(self, i):
        r"""
        Function returning the i'th generator of this elliptic curve.

        """
        return self.gens()[i]
        
    

In [41]:
#MyEllipticCurve_ring
#twist, isogeny, division_field
#category

    
class MyEllipticCurve_ring(MyEllipticCurve_generic):

    def __init__(self, R, data, category=None):
        
        from sage.categories.schemes import Schemes_over_base
        if category is None:
            category = None #Schemes_over_base(R)
        
        super().__init__(R, data, category=category)

    base_ring = MyEllipticCurve_generic.base_ring

    _point = MyEllipticCurve_generic._point

    def genus(self):
        
        return ZZ.one()

  

In [42]:
E = MyEllipticCurve(Zmod(15),[1,2])

In [43]:
E

Elliptic Curve defined by y^2 = x^3 + x + 2 over Ring of integers modulo 15

In [44]:
MyEllipticCurve(QQ,[1,2])

Elliptic Curve defined by y^2 = x^3 + x + 2 over Rational Field

In [45]:
#MyEllipticCurvePoint_ring


class MyEllipticCurvePoint_ring(SchemeMorphism_point_projective_ring):
       
    def __init__(self, curve, v, check=True):
       
        point_homset = curve.point_homset()
        R = point_homset.value_ring()
        if is_SchemeMorphism(v) or isinstance(v, MyEllipticCurvePoint_ring):
            v = list(v)
        elif v == 0:
            v = (R.zero(), R.one(), R.zero())

        SchemeMorphism_point_projective_ring.__init__(self, point_homset, v, check=check)
        AdditiveGroupElement.__init__(self, point_homset)

        self.normalize_coordinates()

    def _repr_(self):
        
        return self.codomain().ambient_space()._repr_generic_point(self._coords)

    def _latex_(self):
        return self.codomain().ambient_space()._latex_generic_point(self._coords)

    def __getitem__(self, n):
        return self._coords[n]

    def __iter__(self):
        
        return iter(self._coords)

    def __tuple__(self):
        """
        Return the coordinates of this point as a tuple.

        EXAMPLES::

            sage: E = EllipticCurve('44a')
            sage: P = E([1, -2, 1])
            sage: P.__tuple__()
            (1, -2, 1)
        """
        return tuple(self._coords)  # Warning: _coords is a list!

    def _richcmp_(self, other, op):
        
        if not isinstance(other, MyEllipticCurvePoint_ring):
            try:
                other = self.codomain().ambient_space()(other)
            except TypeError:
                return NotImplemented
        # op_EQ
        if op == 2:
            return richcmp(self._coords, other._coords, op)

        try:
            return richcmp(self._zxy_coords, other._zxy_coords, op)
        except AttributeError:
            self._zxy_coords = (self._coords[2], self._coords[0], self._coords[1])
            other._zxy_coords = (other._coords[2], other._coords[0], other._coords[1])
            return richcmp(self._zxy_coords, other._zxy_coords, op)

    def __pari__(self):
       
        x,y,z = self._coords
        if z:
            return pari([x/z, y/z])
        else:
            return pari([0])

    def scheme(self):

        return self.codomain()
    

In [46]:
E = MyEllipticCurve(Zmod(15),[1,2])


In [47]:
E

Elliptic Curve defined by y^2 = x^3 + x + 2 over Ring of integers modulo 15

In [48]:
E(0)

(0 : 1 : 0)

In [49]:
P1 = E([5, 3, 10])

In [50]:
P2 = E([10,6,5])

In [51]:
P1 == P2 

True

In [52]:
P = E([5, 2, 5])

In [53]:
Q = E([10,4,10])

In [54]:
P == Q

True

In [55]:
R = E([2, 6, 7])

In [56]:
R2 = E([8, mod(6*4,15) , mod(7*4,15)])

In [57]:
R == R2

True

False